In [2]:
from google.colab import drive
drive.mount('/content/drive/')
data = '/content/drive/My\ Drive/nlp/'
% cd {data}

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/
/content/drive/My Drive/nlp


In [8]:
#from colab_ssh import launch_ssh, init_git
launch_ssh('1e8QobbosS84oTFHpRiZzEjny6V_7fSZ8xRqRRrg4M6woLEMY', 'abab4656')
#!pip install colab_ssh

Successfully running 2.tcp.ngrok.io:12466
[Optional] You can also connect with VSCode SSH Remote extension using this configuration:

	Host google_colab_ssh
		HostName 2.tcp.ngrok.io
		User root
		Port 12466
	  


In [9]:
init_git("https://github.com/Vikramardham/nlp_colab.git",
         #personal_token="<YOUR_GITHUB_PERSONAL_TOKEN>", 
         branch="master",
         email="vikram.ardham@outlook.com",
         username="Vikramardham")

Successfully cloned the repository


In [ ]:
%load_ext autoreload
%autoreload 2
import io
import numpy as np
import pandas as pd
import random
from tqdm.notebook import tqdm 
import wandb

from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split

from nlp_models import *
from nlp_train import *
from torchsummary import summary

from fastai import *
from fastai.text import *
from fastai.tabular import *
from fastai.callbacks import SaveModelCallback, EarlyStoppingCallback
import spacy
#import fasttext as ft

In [ ]:
categories = ['sci.crypt', 'sci.electronics',
              'sci.med', 'sci.space', 'soc.religion.christian']
newsgroups_all= fetch_20newsgroups(subset='all',
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)
X_train, X_test, y_train, y_test = train_test_split(newsgroups_all.data, newsgroups_all.target,
                                         test_size=0.2, stratify=newsgroups_all.target)
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test,
                                         test_size=0.5, stratify=y_test)

> # Tokenize and Create Embedding

In [ ]:
tokenizer = Tokenizer()
spc = SpacyTokenizer('en')
Nwords = 25
data_path=Path('.')
wiki_words = pickle.load(open(data_path / 'itos_wt103.pkl','rb'))
wiki_vocab = Vocab.create([wiki_words], max_vocab=60000, min_freq=1)

def pipeline(xin, yin):
    return mydataset([pad_zeros(wiki_vocab.numericalize(i)) for i in tokenizer.process_all(xin)], yin)

token_train = tokenizer.process_all(X_train)

def pad_zeros(inp, max_len=Nwords):
    ''' pad zeros if the len(input) < max_len'''
    if len(inp)>=max_len:
        return inp[:max_len]
    else:
        return inp+[0]*(max_len-len(inp))
    
xtrain = [pad_zeros(wiki_vocab.numericalize(i)) for i in token_train]
word_to_ix = {word: i for i, word in enumerate(wiki_vocab.itos)}

valid =pipeline(X_valid, y_valid)
test = pipeline(X_test, y_test)
train_data = mydataset(xtrain, y_train)

del X_valid
del X_test
del X_train
del y_train
del xtrain

# Train

In [ ]:
# fix improper tokenization
seq_ln = Nwords
emb_sz = 300
hd_sz = 10
output_sz = 5
vocab_sz = len(wiki_vocab.itos)
bs = 16

In [ ]:
def create_emb(vecs, itos, em_sz=10, mult=1.):
    emb = nn.Embedding(len(itos), em_sz, padding_idx=1)
    wgts = emb.weight.data
    vec_dic = {w:vecs.get_word_vector(w) for w in vecs.get_words()}
    miss = []
    for i,w in enumerate(itos):
        try: wgts[i] = tensor(vec_dic[w])
        except: miss.append(w)
    return emb
#en_vecs = ft.load_model(str(('../../data/external/cc.en.300.bin')))
#emb_enc = create_emb(en_vecs, wiki_vocab.itos)
#torch.load(emb_enc, 'en_emb.pth')
emb_enc = torch.load(data_path/'en_emb.pth')

In [ ]:
rnn_model = SimpleRNN(emb_enc, seq_ln, emb_sz, hd_sz, vocab_sz)
gru_model = SimpleGRU(emb_enc, seq_ln, emb_sz, hd_sz, vocab_sz, bs=bs)
rnn_train = SimpleTrain(rnn_model, nn.CrossEntropyLoss())
gru_train = SimpleTrain(gru_model, nn.CrossEntropyLoss(),)

In [ ]:
# new features with pytorch GRU
# stacked GRU
# dropout
# bidirectional
# bias

In [ ]:
class myGRU(nn.Module):
    def __init__(self, emb_enc, seq_ln, hd_sz, stacks=1):
        super(myGRU, self).__init__()
        self.emb = emb_enc
        self.seq_ln = seq_ln
        self.hd_sz = hd_sz
        self.emb_sz = emb_enc.embedding_dim
        self.vocab_sz = 60000
        self.gru = nn.GRU(input_size=self.emb_sz, 
                             hidden_size=self.hd_sz,
                             num_layers =stacks, bias=True, 
                             dropout=0.2, bidirectional=False, 
                             batch_first=True)
        self.h2o = nn.Linear(self.seq_ln*self.hd_sz, 5)
    def forward(self, X):
        h = self.emb(X)
        gru_out, _ = self.gru(h)
        out= self.h2o(gru_out.reshape(-1, self.seq_ln*self.hd_sz))
        
        return torch.nn.LogSoftmax(dim=0)(out)

In [ ]:
default_config = dict({'lr' : 0.005, 'epochs':10, 'bs': 16, 'wd':1e-4, 'stacks':1, 'hd_sz':10})
sweep_config = {
    'method': 'grid', # random
    'metric': {
      'name': 'acc',
      'goal': 'maximize'   
    },
    'parameters': {
        #'epochs': {'values': [10]},
        #'lr': {'values': [1e-2, 1e-3, 1e-4, 1e-5, 1e-6]},
        'wd': {'values': [1e-03, 1e-06, 1e-4, 1e-2]},
        'bs' : {'values': [8, 64, 1024]},
        'stacks':{'values':[1, 2, 5, 10]},
        'hd_sz':{'values': [10, 100, 300, 500]},
    },
}

In [ ]:
for wd in [1e-05, 1e-03, 1e-1]:
    for bs in [8, 64, 128]:
        for stacks in [1, 10]:
            for hd_sz in [300, 1000]:
                default_config = dict({'lr' : 0.005, 'epochs':10, 
                                       'bs': bs, 'wd':wd, 
                                       'stacks':stacks, 'hd_sz':hd_sz})
                wandb.init(config=default_config, project='nlp_basics')
                config = wandb.config
                gru = myGRU(emb_enc, seq_ln, config.hd_sz, config.stacks)
                train = SimpleTrain(gru, nn.CrossEntropyLoss())
                train.train(train_data, valid, config)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type myGRU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


--------------------------------------------
Epoch:	 Train_loss 	 Valid_loss	acc	precision	recall	
0	 2.3969 	 3.6939 	 0.3273	0.3420	0.3273	
---------------------------------------------
--------------------------------------------
Epoch:	 Train_loss 	 Valid_loss	acc	precision	recall	
1	 73.5624 	 115.6753 	 0.2141	0.2134	0.2141	
---------------------------------------------
--------------------------------------------
Epoch:	 Train_loss 	 Valid_loss	acc	precision	recall	
2	 230.3405 	 176.5998 	 0.2586	0.3001	0.2586	
---------------------------------------------
--------------------------------------------
Epoch:	 Train_loss 	 Valid_loss	acc	precision	recall	
3	 239.3846 	 221.7980 	 0.2869	0.3296	0.2869	
---------------------------------------------
--------------------------------------------
Epoch:	 Train_loss 	 Valid_loss	acc	precision	recall	
4	 177.0157 	 213.6288 	 0.3010	0.3331	0.3010	
---------------------------------------------
--------------------------------------------


--------------------------------------------
Epoch:	 Train_loss 	 Valid_loss	acc	precision	recall	
0	 49.1832 	 36.1380 	 0.2384	0.2566	0.2384	
---------------------------------------------
--------------------------------------------
Epoch:	 Train_loss 	 Valid_loss	acc	precision	recall	
1	 263.5151 	 205.0415 	 0.2667	0.2746	0.2667	
---------------------------------------------
--------------------------------------------
Epoch:	 Train_loss 	 Valid_loss	acc	precision	recall	
2	 349.2090 	 391.3896 	 0.2444	0.2642	0.2444	
---------------------------------------------
--------------------------------------------
Epoch:	 Train_loss 	 Valid_loss	acc	precision	recall	
3	 538.3912 	 420.3998 	 0.2444	0.2661	0.2444	
---------------------------------------------
--------------------------------------------
Epoch:	 Train_loss 	 Valid_loss	acc	precision	recall	
4	 371.7216 	 500.5337 	 0.2566	0.3155	0.2566	
---------------------------------------------
------------------------------------------

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


--------------------------------------------
Epoch:	 Train_loss 	 Valid_loss	acc	precision	recall	
0	 1.6228 	 1.6094 	 0.1960	0.0389	0.1960	
---------------------------------------------
--------------------------------------------
Epoch:	 Train_loss 	 Valid_loss	acc	precision	recall	
1	 1.6095 	 1.6094 	 0.2061	0.1070	0.2061	
---------------------------------------------
--------------------------------------------
Epoch:	 Train_loss 	 Valid_loss	acc	precision	recall	
2	 104.1936 	 39.5395 	 0.1879	0.2718	0.1879	
---------------------------------------------
--------------------------------------------
Epoch:	 Train_loss 	 Valid_loss	acc	precision	recall	
3	 11.8680 	 7.0712 	 0.1919	0.2411	0.1919	
---------------------------------------------
--------------------------------------------
Epoch:	 Train_loss 	 Valid_loss	acc	precision	recall	
4	 6.8788 	 5.1844 	 0.1859	0.2200	0.1859	
---------------------------------------------
--------------------------------------------
Epoch:	 Tra

In [ ]:
def train_gru():
    wandb.init(config = default_config)
    config = wandb.config
    gru = myGRU(emb_enc, seq_ln, config.hd_dz, config.stacks)
    train = SimpleTrain(gru, nn.CrossEntropyLoss())
    train.train(train_data, valid, config)
    return 

In [ ]:
#sweep_id = wandb.sweep(sweep_config)
#wandb.sweep(sweep_id, function=train_gru)

In [ ]:
!pip install wandb

     |████████████████████████████████| 1.4MB 8.5MB/s 
     |████████████████████████████████| 102kB 13.3MB/s 
     |████████████████████████████████| 102kB 12.7MB/s 
     |████████████████████████████████| 112kB 36.3MB/s 
     |████████████████████████████████| 460kB 26.7MB/s 
     |████████████████████████████████| 71kB 10.6MB/s 
     |████████████████████████████████| 71kB 11.2MB/s 
  Created wheel for gql: filename=gql-0.2.0-cp36-none-any.whl size=7630 sha256=ec0ebf7149df55e50c8a5924dede9b1806a3b9855eaa58b72c0af73fc1133c6c
  Stored in directory: /root/.cache/pip/wheels/ce/0e/7b/58a8a5268655b3ad74feef5aa97946f0addafb3cbb6bd2da23
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=b502dfc35997df917c61d5a53ef763e4fa23ec400c1b60a9a27b3c815efb0d83
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for watchdog: filename=watchdog-0.10.2-cp36-none-any.whl size=73605 sha256=

In [ ]:
wandb.init(project='gru', config=default_config)
wandb.watch(gru_model)

In [ ]:
sweep_config = {
    'method': 'grid' # random
    'metric': {
      'name': 'acc',
      'goal': 'maximize'   
    },
    'parameters': {
        #'epochs': {'values': [10]},
        #'lr': {'values': [1e-2, 1e-3, 1e-4, 1e-5, 1e-6]},
        'wd': {'values': [1e-03, 1e-06, 1e-4, 1e-2]},
        'bs' : {'values': [8, 64, 1024]},
        'stack':{'values':[1, 2, 5, 10]}
    }
}
sweep_id = wandb.sweep(sweep_config)

In [ ]:
def train():
    gru_train.train(train_data, valid, default_config)
    return

In [ ]:
#gru_train.train(train_data, valid, default_config)

In [ ]:
## improve training --- 90%?
## sweep with wandb

## rnn v gru
## write an article
## create networks? pandas networkx

## move on to seq2seq